In [1]:
from CoolProp.CoolProp import FluidsList
FluidsList()
from CoolProp.CoolProp import PhaseSI, PropsSI, get_global_param_string
import CoolProp.CoolProp as CoolProp
from CoolProp import AbstractState
import pandas as pd

In [2]:
to_kelvin = lambda x: x + 273.15 if x else None
type_sheet = 'Н'
#Нагреваемый поток 
p0_heated_flow = 5
t0_heated_flow = 30
G_heated_flow = 10
fluid_heated_flow  = 'water'
params_heated_flow = {
    'P':p0_heated_flow* 10**6,
    'T':to_kelvin(t0_heated_flow),
    'H':PropsSI("H", "P", p0_heated_flow * 10**6, "T", to_kelvin(t0_heated_flow), fluid_heated_flow),
    'S': PropsSI("S", "P", p0_heated_flow * 10**6, "T", to_kelvin(t0_heated_flow), fluid_heated_flow),
    'fluid': fluid_heated_flow,
    'G': G_heated_flow
    
}
#Греюший поток
p0_warming_flow = 10
t0_warming_flow = 200
G_warming_flow = 10
fluid_warming_flow  = 'water'


params_warming_flow = {
    'P':p0_warming_flow* 10**6,
    'T':to_kelvin(t0_warming_flow),
    'H':PropsSI("H", "P", p0_warming_flow * 10**6, "T", to_kelvin(t0_warming_flow), fluid_warming_flow),
    'S': PropsSI("S", "P", p0_warming_flow * 10**6, "T", to_kelvin(t0_warming_flow), fluid_warming_flow),
    'fluid': fluid_warming_flow,
    'G': G_warming_flow
}

In [3]:
from typing import Optional, Tuple, List, Union, Dict
from dataclasses import dataclass, field
import numpy as np
from CoolProp.CoolProp import FluidsList
from CoolProp.CoolProp import PhaseSI, PropsSI, get_global_param_string
import CoolProp.CoolProp as CoolProp


@dataclass
class Surface_Heat_Exchanger:
    
    params_heated_flow: Dict[str, Union[int,float]] =  field(default_factory=lambda:dict)
    params_warming_flow: Dict[str, Union[int,float]] =  field(default_factory=lambda:dict)
    underheating: float = field(default=15)
    type_sheet: str = field(default='Н')
    
    @property
    def t_heated_flow_out(self)-> float:
        """
        Температура нагреваемого потока на выходе 
        """
        return  self.params_warming_flow['T'] - self.underheating
    
    @property
    def heat_flow(self)-> float:
        """
        Теловой поток Q
        """
        return self.params_heated_flow['G'] * (PropsSI("H", "P", self.params_heated_flow['P'], "T", self.t_heated_flow_out, self.params_heated_flow['fluid']) - self.params_heated_flow['H'])

    @property
    def t_warming_flow_out(self)-> float:
        """
        температура греюшего потка на выходе 
        """
        return  self.params_warming_flow['T'] - (self.heat_flow / (self.params_warming_flow['G'] * PropsSI("C", "P", self.params_warming_flow['P'], "T", self.params_warming_flow['T'], self.params_warming_flow['fluid'])))
    
    @property
    def aver_log_t_dif(self)-> float:
        """
        Средняя логорифмическая температура
        """
        delta_t_b = self.params_warming_flow['T'] - self.t_heated_flow_out
        delta_t_m = -self.params_heated_flow['T'] + self.t_warming_flow_out
        return (delta_t_b - delta_t_m) / (np.log(delta_t_b/delta_t_m)) 
    
    @property
    def heat_transfer_coefficient(self)-> float:
        """
        Коэффициент теплопередачи для двух теплоносителей 
        """
        l1 = pd.read_excel('book.xlsx',sheet_name ='l1')
        kop  = l1.loc[((l1['heated_flow'] == self.params_heated_flow["fluid"]) & 
                       (l1['warming_flow'] == self.params_warming_flow["fluid"])),['kop']]
        return float(kop.values)
    
    @property
    def approximate_heat_transfer_surface(self)->float:
        """
        Ориентировочная поверхность теплообмена
        """
        return self.heat_flow / (self.heat_transfer_coefficient * self.aver_log_t_dif)
    
    
    @property
    def heat_flux_density(self)->float: 
        """
        Плотность теплового потока
        """
        return self.heat_flow / self.approximate_heat_transfer_surface
        
    @property
    def params_of_sheet(self)->float:
        """
        Параметры ГОСТ пластин 
        """
        l2 = pd.read_excel('book.xlsx',sheet_name ='l2')
        if self.type_sheet == 'Н':
            F_  = l2.loc[((self.approximate_heat_transfer_surface + 250 >= l2['Н']) &
                         (self.approximate_heat_transfer_surface - 250 <= l2['Н'])),
                         ['Площадь поверхности теплообмена пластины, м2']]
            F_ = float(F_.values[-1])
        else:
            F_ = l2.loc[((l2['Р'] >= a.approximate_heat_transfer_surface - 50) &
                         (l2['Р'] <= a.approximate_heat_transfer_surface + 50) |
                         (l2['Р.1'] >= a.approximate_heat_transfer_surface - 50) &
                         (l2['Р.1'] <= a.approximate_heat_transfer_surface + 50) |
                         (l2['Р.2'] >= a.approximate_heat_transfer_surface - 50) & 
                         (l2['Р.2'] <= a.approximate_heat_transfer_surface + 50) |   
                         (l2['РС'] >= a.approximate_heat_transfer_surface - 50 ) &
                         (l2['РС'] <= a.approximate_heat_transfer_surface + 50) |
                         (l2['РС.1'] >= a.approximate_heat_transfer_surface - 50) & 
                         (l2['РС.1'] <= a.approximate_heat_transfer_surface + 50) |
                         (l2['РС.2'] >= a.approximate_heat_transfer_surface - 50) &
                         (l2['РС.2'] <= a.approximate_heat_transfer_surface + 50)),
                         ['Площадь поверхности теплообмена пластины, м2']]
            F_ = float(F_.values[-1])
        l3 = pd.read_excel('book.xlsx',sheet_name ='l3')

        params_sheet = l3.loc[((F_ + 0.05 >= l3['Площадь поверхности теплообмена F, м2']) &
                 (F_ - 0.05 <= l3['Площадь поверхности теплообмена F, м2'])&
                  (self.type_sheet == l3['Тип']))]
        return params_sheet
    
    @property
    def Re_number(self)->float:
        """
        Число Рейнольдса
        """
        d = float(pd.DataFrame(self.params_of_sheet,columns = ['Площадь поперечного сечения одного канала 102 м2']).values)
        L = float(pd.DataFrame(self.params_of_sheet,columns = ['Приведенная длинна канала Lп, м']).values)
        D = float(pd.DataFrame(self.params_of_sheet,columns = ['Эквивалентный деаметр dэ, 102 м2']).values)
        if self.params_warming_flow['fluid'] == 'steam':  
            return (self.heat_flow * L) / (2160000 * PropsSI("V", "P", self.params_warming_flow['P'], "T", self.t_warming_flow_out, 'water'))
        else:
            return (self.params_warming_flow['G']  * L ) / (PropsSI("V", "P", self.params_warming_flow['P'], "T", self.t_warming_flow_out, self.params_warming_flow['fluid']) * d)
    
    @property        
    def Nu_number(self)->float: 
        l4 = pd.read_excel('book.xlsx',sheet_name ='l4')
        F = float(pd.DataFrame(self.params_of_sheet,columns = ['Площадь поверхности теплообмена F, м2']).values)
        Pr = PropsSI("Prandtl", "P", p0_warming_flow * 10**6, "T", to_kelvin(t0_warming_flow), fluid_warming_flow)
        tcp = (((self.params_warming_flow['T'] + self.t_warming_flow_out)/2) + ((self.t_heated_flow_out + self.params_heated_flow['T'])/2))
        Prc = PropsSI("Prandtl", "P", p0_warming_flow * 10**6, "T", tcp, fluid_warming_flow)
        if self.params_warming_flow['fluid'] == 'steam':  
            coef = l4.loc[((l4['Тип'] == self.type_sheet) & (l4['Площадь поверхности теплообмена F, м2'] == F)),['Ск',]]
            Nu = float(coef.values) * self.Re_number ** 0.7 * Pr ** 0.4
        else:
            if self.Re_number > 10**4:  # турбулентный режим
                coef = l4.loc[((l4['Тип'] == self.type_sheet) & (l4['Площадь поверхности теплообмена F, м2'] == F)),['Ст','Ат']]
                Nu = coef.values[0][0] * self.Re_number ** 0.73 * Pr ** 0.43 * (Pr/Prc) ** 0.25
            else:
                coef = l4.loc[((l4['Тип'] == self.type_sheet) & (l4['Площадь поверхности теплообмена F, м2'] == F)),['Сл','Ал']]
                Nu = coef.values[0][0] * self.Re_number ** 0.33 * Pr ** 0.43 * (Pr/Prc) ** 0.25    
        return Nu

In [4]:
a = Surface_Heat_Exchanger(params_heated_flow = params_heated_flow, params_warming_flow = params_warming_flow)

In [5]:
a.Nu_number

178.99459543085152